In [114]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re 
import seaborn as sns
from collections import ChainMap
from collections import Counter
from itertools import combinations
from scipy.stats import ttest_ind
from sklearn import metrics # инструменты для оценки точности модели  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn.model_selection import train_test_split  # Загружаем специальный инструмент для разбивки: 
from statsmodels.stats import weightstats

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

In [69]:
main_task = pd.read_csv('main_task.csv')
# для удобства переименуем переменные
main_task.columns = ['restaurant_id', 'city', 'cuisine_style', 'ranking', 'rating',
              'price_range', 'number_of_reviews', 'reviews', 'URL_TA', 'ID_TA']

In [29]:
#  предобработка

df = main_task.copy()

# для удобства переименуем переменные
# df.columns = ['restaurant_id', 'city', 'cuisine_style', 'ranking', 'rating',
#        'price_range', 'number_of_reviews', 'reviews', 'URL_TA', 'ID_TA']


*  Restaurant_id — идентификационный номер ресторана / сети ресторанов;
*   City — город, в котором находится ресторан;
*  Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
*  Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
* Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
*    Price Range — диапазон цен в ресторане;
*   Number of Reviews — количество отзывов о ресторане;
*  Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
* URL_TA — URL страницы ресторана на TripAdvisor;
* ID_TA — идентификатор ресторана в базе данных TripAdvisor.


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      40000 non-null  object 
 1   city               40000 non-null  object 
 2   cuisine_style      30717 non-null  object 
 3   ranking            40000 non-null  float64
 4   rating             40000 non-null  float64
 5   price_range        26114 non-null  object 
 6   number_of_reviews  37457 non-null  float64
 7   reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [16]:
df.sample(5)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA
1957,id_2489,Berlin,"['Chinese', 'Asian']",2491.0,4.0,$,21.0,"[['Second Time, Best time!', 'Authenic Chinese...",/Restaurant_Review-g187323-d7158748-Reviews-As...,d7158748
23347,id_4239,Milan,"['Italian', 'Pizza']",4242.0,3.5,NaN,44.0,"[['For Milano mid range food, go here', 'it is...",/Restaurant_Review-g187849-d2299261-Reviews-Fa...,d2299261
34268,id_2362,Munich,['Diner'],2379.0,3.5,$$ - $$$,7.0,[['Good overnight stop during travel to Austr....,/Restaurant_Review-g187309-d964492-Reviews-Neu...,d964492
32872,id_1768,Vienna,['Italian'],1769.0,5.0,NaN,NaN,"[[], []]",/Restaurant_Review-g190454-d5961438-Reviews-Re...,d5961438
30280,id_3125,Paris,"['Italian', 'Vegetarian Friendly']",3126.0,4.0,$$ - $$$,69.0,"[['Great food, great price', 'Dinner'], ['10/0...",/Restaurant_Review-g187147-d11844052-Reviews-C...,d11844052


In [23]:
type(df.loc[0,'reviews'])
# data.loc[0,'reviews']

str

На первом этапе для создания такого датафрейма давайте просто удалим столбцы, содержащие данные типа __object__, и заполним пропущенные значения (__None__ или __NaN__) каким-то одним значением (нулём или средним арифметическим) для всего столбца.

In [57]:
display(df.info())
df[df.number_of_reviews.isna()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ranking            40000 non-null  float64
 1   rating             40000 non-null  float64
 2   number_of_reviews  37457 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


None

,ranking,rating,number_of_reviews
8,2692.0,4.0,NaN
21,5847.0,4.0,NaN
32,1328.0,5.0,NaN
102,1410.0,5.0,NaN
108,2050.0,5.0,NaN
...,...,...,...
39965,5722.0,5.0,NaN
39972,2012.0,5.0,NaN
39986,7279.0,3.0,NaN
39988,4970.0,4.0,NaN


In [59]:
df = df.drop([column  for column in df.columns if df[column].dtype == 'O'],axis=1)
df.number_of_reviews = df.number_of_reviews.fillna(0)
display(df.info())
df[df.number_of_reviews.isna()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ranking            40000 non-null  float64
 1   rating             40000 non-null  float64
 2   number_of_reviews  40000 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


None

,ranking,rating,number_of_reviews


In [61]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
# X = df.drop(['restaurant_id', 'rating'], axis = 1)  
X = df.drop(['rating'], axis = 1)  

y = df['rating']  
      
    # Загружаем специальный инструмент для разбивки:  убрано наверх
#     from sklearn.model_selection import train_test_split   
    
      
    # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
    # Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    

In [62]:
# Импортируем необходимые библиотеки: убрано наверх 
#     from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
#     from sklearn import metrics # инструменты для оценки точности модели  
      
    # Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
    # Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
    # Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

In [64]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются  
    # Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.  
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  

MAE: 0.43295923492063493


 4.1 Вопросы о ценах

Сколько вариантов непустых значений встречается в столбце Price Range?

In [129]:
df_4 = main_task.copy()

In [102]:
variants = [element for element in(df_4.price_range.unique()) if type(element) == str]
display(len(variants),variants)


3

['$$ - $$$', '$$$$', '$']

In [72]:
df_4.sample()

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA
30706,id_6956,Barcelona,"['Fast Food', 'Turkish', 'Grill']",6957.0,3.0,NaN,10.0,"[['The food is good...', 'Doner makes beautifu...",/Restaurant_Review-g187497-d6539515-Reviews-Re...,d6539515


Сколько ресторанов относятся к среднему ценовому сегменту?

In [75]:
len(df_4[df_4.price_range == '$$ - $$$'])

18412

 4.2 Вопрос о городах  
1 point possible (graded)  
Сколько городов представлено в наборе данных?

In [105]:
len(df_4.city.unique())

31

4.3 Вопросы о кухнях

Сколько типов кухонь представлено в наборе данных?

In [170]:
df_4 = main_task.copy()
df_4.cuisine_style = df_4.cuisine_style.str.replace("'", '')
df_4.cuisine_style = df_4.cuisine_style.str.replace("[", '')
df_4.cuisine_style = df_4.cuisine_style.str.replace("]", '')
df_4.cuisine_style = df_4.cuisine_style.str.replace(" ", '')
df_4.cuisine_style = df_4.cuisine_style.str.split(',')
df_4 = df_4.explode('cuisine_style')


In [169]:
# не работает пока что
df_4 = main_task.copy()
df_4.cuisine_style = df_4.cuisine_style.str.strip("'' []")

# df_4.cuisine_style = df_4.cuisine_style.str.strip()
# df_4.cuisine_style = df_4.cuisine_style.str.split(',')
# df_4 = df_4.explode('cuisine_style')
display(type(df_4.cuisine_style.loc[0]))
df_4.cuisine_style.loc[0]

str

"European', 'French', 'International"

In [171]:
cuisine = [element for element in (df_4.cuisine_style.unique()) if type(element) == str]
display(len(cuisine))
df_4.cuisine_style.value_counts()

125

VegetarianFriendly    11189
European              10060
Mediterranean          6277
Italian                5964
VeganOptions           4486
                      ...  
Burmese                   1
Yunnan                    1
Latvian                   1
Xinjiang                  1
Salvadoran                1
Name: cuisine_style, Length: 125, dtype: int64

Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. Ответ округлите до одного знака после запятой.

In [144]:
df4 = main_task.copy()
df4.cuisine_style = df4.cuisine_style.str.replace("'", '')
df4.cuisine_style = df4.cuisine_style.str.replace("[", '')
df4.cuisine_style = df4.cuisine_style.str.replace("]", '')
df4.cuisine_style = df4.cuisine_style.str.replace(" ", '')
df4.cuisine_style = df4.cuisine_style.str.split(',')

def return_count(value):
    if type(value) == list:
        return len(value)
    else: return 1
df4['cuisine_number'] = df4.cuisine_style.apply(return_count)
round(df4.cuisine_number.mean(),1)

2.6